# Answering business questions using SQL

First we import the database.

In [19]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

In [20]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");


Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


There are several tables containing information about the transactions made in a music store called "Chinook".

## Selecting Albums to Purchase

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce.

- TASK: You'll need to write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store.

In [21]:
%%sql
DROP VIEW invoice2;
CREATE VIEW invoice2 AS
SELECT c.customer_id,i.invoice_id
            FROM customer c
            LEFT JOIN invoice i ON i.customer_id=c.customer_id
            WHERE c.country='USA';



Done.
Done.


[]

In [22]:
%%sql


SELECT g.name genre,COUNT(g.name) number_sold, COUNT(g.name)*100/(SELECT COUNT(track_id) FROM invoice2 i2
LEFT JOIN invoice_line il ON il.invoice_id=i2.invoice_id) percentage
FROM invoice2 i2
LEFT JOIN invoice_line il ON i2.invoice_id=il.invoice_id
LEFT JOIN track t ON il.track_id=t.track_id
LEFT JOIN genre g ON t.genre_id=g.genre_id
GROUP BY g.name
ORDER BY COUNT(g.name) DESC

Done.


genre,number_sold,percentage
Rock,561,53
Alternative & Punk,130,12
Metal,124,11
R&B/Soul,53,5
Blues,36,3
Alternative,35,3
Latin,22,2
Pop,22,2
Hip Hop/Rap,20,1
Jazz,14,1


It seems that not all hope is lost and the Chinook customers favourite genres are Rock, Alternative & Punk and Metal. The data indicates that from the 4 given albums the best candidates are "Red Tone"(Punk), "Slim Jim Bites (Blues)" and "Meteor and the Girls (Pop)". It would be wise to consider including more Rock albums since they make up 53% of the purchases.

## Analyzing Employee Sales Performance

- TASK: 
        1.Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you find are relevant to the analysis.

        2.Write a short statement describing your results, and providing a possible interpretation.

In [23]:
%%sql
WITH customertotal AS(
SELECT i.customer_id,SUM(i.total) total,c.support_rep_id
FROM invoice i
LEFT JOIN customer c ON i.customer_id=c.customer_id
GROUP BY 1)
SELECT support_rep_id employee, SUM(total) total_sales,e.hire_date
FROM customertotal ct
INNER JOIN employee e ON e.employee_id=ct.support_rep_id
GROUP BY 1



Done.


employee,total_sales,hire_date
3,1731.5099999999998,2017-04-01 00:00:00
4,1584.0000000000002,2017-05-03 00:00:00
5,1393.92,2017-10-17 00:00:00


While there are some differences in the performance of employees, they might be due to the time that they have been in the company.

## Analyzing Sales by Country

- TASK: Write a query that collates data on purchases from different countries.
Where a country has only one customer, collect them into an "Other" group.
The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.
    - For each country, include:
        - total number of customers
        - total value of sales
        - average value of sales per customer
        - average order value

In [24]:
%%sql

DROP VIEW total_per_id_country;
CREATE VIEW total_per_id_country AS
SELECT c.customer_id,
CASE
WHEN (SELECT count(*) 
      FROM customer
      WHERE country=c.country)=1 THEN 'Other'
      ELSE c.country
      END AS country,
i.total cust_total,
i.total purchase,
i.invoice_id

FROM customer c
LEFT JOIN invoice i ON c.customer_id=i.customer_id
ORDER BY country


Done.
Done.


[]

In [25]:
%%sql
DROP VIEW report;
CREATE VIEW report AS
SELECT 
ttid.country country,
COUNT(distinct ttid.customer_id) num_customers,
SUM(ttid.cust_total) total_sales,
SUM(ttid.cust_total)/COUNT(distinct ttid.customer_id) avg_per_cust,
SUM(ttid.cust_total)/COUNT(distinct ttid.invoice_id) avg_order



FROM total_per_id_country ttid

GROUP BY 1
ORDER BY num_customers DESC

Done.
Done.


[]

In [26]:
%%sql
SELECT *
FROM (
SELECT *,CASE
             WHEN report.country='Other' THEN 1
             ELSE 0
             END AS sort
FROM report)
ORDER BY sort ASC


Done.


country,num_customers,total_sales,avg_per_cust,avg_order,sort
USA,13,1040.4900000000005,80.03769230769234,7.942671755725194,0
Canada,8,535.59,66.94875,7.047236842105264,0
Brazil,5,427.6799999999999,85.53599999999997,7.011147540983605,0
France,5,389.07000000000005,77.81400000000001,7.781400000000001,0
Germany,4,334.61999999999995,83.65499999999999,8.161463414634145,0
United Kingdom,3,245.52,81.84,8.768571428571429,0
Czech Republic,2,273.23999999999995,136.61999999999998,9.107999999999999,0
India,2,183.14999999999995,91.57499999999997,8.72142857142857,0
Portugal,2,185.13,92.565,6.3837931034482756,0
Other,15,1094.94,72.99600000000001,7.448571428571429,1


## Albums vs Individual Tracks

TASK:
- Write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:
    - Number of invoices
    - Percentage of invoices
- Write one to two sentences explaining your findings, and making a prospective recommendation on whether the Chinook store should continue to buy full albums from record companies

In [27]:
%%sql

CREATE VIEW album_tracklist AS
SELECT t.album_id,t.track_id
FROM track t
ORDER BY 1

(sqlite3.OperationalError) table album_tracklist already exists
[SQL: CREATE VIEW album_tracklist AS
SELECT t.album_id,t.track_id
FROM track t
ORDER BY 1]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [28]:
%%sql
CREATE VIEW info AS
SELECT il.invoice_id,t.track_id,t.album_id
FROM invoice_line il
LEFT JOIN track t on t.track_id=il.track_id;


(sqlite3.OperationalError) table info already exists
[SQL: CREATE VIEW info AS
SELECT il.invoice_id,t.track_id,t.album_id
FROM invoice_line il
LEFT JOIN track t on t.track_id=il.track_id;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [29]:
%%sql
CREATE VIEW invoice_first_track AS 
SELECT
    il.invoice_id AS invoice_id,
    MIN(il.track_id) AS first_track_id
FROM invoice_line il

GROUP BY 1



(sqlite3.OperationalError) table invoice_first_track already exists
[SQL: CREATE VIEW invoice_first_track AS 
SELECT
    il.invoice_id AS invoice_id,
    MIN(il.track_id) AS first_track_id
FROM invoice_line il

GROUP BY 1]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [30]:
%%sql
CREATE VIEW megatable AS
SELECT il.invoice_id,at.track_id,at.album_id
FROM invoice_line il
LEFT JOIN album_tracklist at ON at.track_id=il.track_id
ORDER BY 1;




(sqlite3.OperationalError) table megatable already exists
[SQL: CREATE VIEW megatable AS
SELECT il.invoice_id,at.track_id,at.album_id
FROM invoice_line il
LEFT JOIN album_tracklist at ON at.track_id=il.track_id
ORDER BY 1;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [44]:
%%sql
WITH invoice_first_track AS (
  SELECT
    il.invoice_id AS invoice_id,
    MIN(il.track_id) AS first_track_id
  FROM
    invoice_line il
  GROUP BY
    1
)


SELECT
  album_purchase,
  COUNT(invoice_id) AS number_of_invoices,
  CAST(COUNT(invoice_id) AS FLOAT) / (
    SELECT COUNT(*) FROM invoice
  ) AS percent
FROM
  (
    SELECT
      ifs.*,
      CASE
       
        WHEN (
          SELECT
            t.track_id
          FROM
            track t
          WHERE
            t.album_id = (
              SELECT
                t2.album_id
              FROM
                track t2
              WHERE
                t2.track_id = ifs.first_track_id
            )
          EXCEPT
          SELECT
            il2.track_id
          FROM
            invoice_line il2
          WHERE
            il2.invoice_id = ifs.invoice_id
        ) IS NULL
        AND (
          SELECT
            il2.track_id
          FROM
            invoice_line il2
          WHERE
            il2.invoice_id = ifs.invoice_id
          EXCEPT
          SELECT
            t.track_id
          FROM
            track t
          WHERE
            t.album_id = (
              SELECT
                t2.album_id
              FROM
                track t2
              WHERE
                t2.track_id = ifs.first_track_id
            )
        ) IS NULL
        THEN "yes"
        ELSE "no"
      END AS album_purchase
    FROM
      invoice_first_track ifs
  ) subquery

GROUP BY
  album_purchase;


Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue
